In [ ]:
from agents.renderer.test_prompt import final_report_dummy,json_data
from utils.llm import LLMProfile
from langchain_core.output_parsers import JsonOutputParser
from prompts import PromptManager, PromptType
from utils.util import get_today_str
from utils.llm import LLMProfile
from langchain_core.messages import SystemMessage, HumanMessage

llm = LLMProfile.dev_llm()
llm_result = llm.invoke(
    PromptManager(PromptType.RENDERER_SUMMARY_TITLE)
    .get_prompt(final_report=final_report_dummy)
)
summary_title = llm_result.content
prompt_system = PromptManager(PromptType.RENDERER_SYSTEM).get_prompt()
prompt_human = PromptManager(PromptType.RENDERER_HUMAN).get_prompt(
    date=get_today_str(),
    title="서울특별시 영등포구 신길동\n타입: 84제곱미터, 세대수: 1000 사업 보고서",
    summary_title=summary_title,
    final_report=final_report_dummy,
)
slice_plan_llm = LLMProfile.renderer_llm().invoke([
    SystemMessage(content=prompt_system),
    HumanMessage(content=prompt_human)
])


In [2]:
slice_plan_json = JsonOutputParser().parse(slice_plan_llm.content)

In [3]:
slice_plan_json

# import json 
# import ast
# data = ast.literal_eval(json_data03)


{'meta': {'title': '서울특별시 영등포구 신길동 사업 보고서',
  'theme': 'dark_gold',
  'renderer': 'google_slides_simple',
  'compat': ['title', 'bluf', 'text', 'table']},
 'slides': [{'layout_type': 1,
   'type': 'cover',
   'title': '서울특별시 영등포구 신길동\n84㎡ 1,000세대 사업 보고서',
   'lead': '역삼동 84㎡ 분양가 권장범위 및 분양성 핵심 인사이트',
   'org': '부동산 마케팅 협회',
   'date': '2025.11.06',
   'note': '※ 본 보고서는 내부 검토용입니다.'},
  {'layout_type': 2,
   'type': 'text',
   'title': '1. 사업환경 분석',
   'lead': '교통·교육·생활편의 3요건 충족 시 상단가 정당화 가능',
   'groups': [{'label': '교육환경(학군·학원가 접근성)',
     'insight': '중·고 학령층 유지로 학군 선호형 수요 견조',
     'details': {'data': ['10~19세 64,171명(2025.09, +597명, +0.9%)',
       '0~9세 32,142명(2025.09, -628명, -1.9%)',
       '대치·역삼 학군 생활권 인접(실보행 거리 미확인)'],
      'analysis': ['중·고 학령층 증가로 학군 접근성 프리미엄 정당화',
       '실보행 기준 대치·역삼 학군 15분 내 입증 시 84㎡ 가족형 수요 집중',
       '통학로 안전성·고도차 등 현장 검증 필요']}},
    {'label': '교통접근성(역세권·환승 연결)',
     'insight': '도보 7~10분 역세권 확보 시 상단가 정당화',
     'details': {'data': ['최근접 지하철역(2·9·수인분당선) 실

In [ ]:
# from agents.renderer.renderer_logic import render_sliceplan_local
# path = render_sliceplan_local(slice_plan_json)

# from tools.send_gmail import gmail_authenticate
# from tools.send_gmail import send_gmail
# gmail_authenticate()

# # to="immortal0900@gmail.com",
# send_gmail(
#         md_content="test",
#         to="dlwlstjq410@gmail.com",
#         title="test",
#         attachment_path=path
#     )

PPTX saved: /Users/seobi/PythonProjects/RAG_Commander/src/agents/renderer/aa1ece4ff71b4375bed6bd40cbb7ab18.pptx
📎 첨부파일 추가 완료: aa1ece4ff71b4375bed6bd40cbb7ab18.pptx
✅ HTML 본문 메일 전송 완료! ID: 19a54af1f3ea31a6
